In [1]:
import keras
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf1
# # set this so LinearModel.fit won't have bug
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

#### Prepare data for model

In [2]:
x_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
x_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

In [3]:
# 将类别转为数字
x_train['sex'].replace(('male', 'female'), (0, 1), inplace=True)
x_eval['sex'].replace(('male', 'female'), (0, 1), inplace=True)

x_train['alone'].replace(('n', 'y'), (0, 1), inplace=True)
x_eval['alone'].replace(('n', 'y'), (0, 1), inplace=True)

x_train['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)
x_eval['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)

# 将没用的列Drop
x_train.drop(['embark_town', 'deck'], axis=1, inplace=True)
x_eval.drop(['embark_town', 'deck'], axis=1, inplace=True)

# 将target单独拿出
y_train = x_train.pop('survived')
y_eval = x_eval.pop('survived')

In [4]:
# Data setup for TensorFlow 1 with `tf.estimator`
def _input_fn():
    return tf1.data.Dataset.from_tensor_slices((dict(x_train), y_train)).batch(32)


def _eval_input_fn():
    return tf1.data.Dataset.from_tensor_slices((dict(x_eval), y_eval)).batch(32)


FEATURE_NAMES = [
    'age', 'fare', 'sex', 'n_siblings_spouses', 'parch', 'class', 'alone'
]

feature_columns = []
for fn in FEATURE_NAMES:
    feat_col = tf1.feature_column.numeric_column(fn, dtype=tf.float32)
    feature_columns.append(feat_col)

In [5]:
# Create a method to instantiate a simplistic sample optimizer to use with our various TensorFlow 1 Estimator and TensorFlow 2 Keras models

def create_sample_optimizer(tf_version):
    if tf_version == 'tf1':
        optimizer = lambda: tf.keras.optimizers.Ftrl(
            l1_regularization_strength=0.001,
            learning_rate=tf1.train.exponential_decay(
                learning_rate=0.1,
                global_step=tf1.train.get_global_step(),
                decay_steps=10000,
                decay_rate=0.9))
    elif tf_version == 'tf2':
        optimizer = tf.keras.optimizers.Ftrl(
            l1_regularization_strength=0.001,
            learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=0.1, decay_steps=10000, decay_rate=0.9))
    return optimizer


#### Create Model

##### Train Model SEPARATELY before JOINTLY Training

In [17]:
?? tf.keras.losses

In [ ]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

In [24]:
linear_model = tf.keras.experimental.LinearModel()
linear_model.compile('adagrad', 'mse')
linear_model.fit(x_train, y_train, epochs=10)
dnn_model = tf.keras.Sequential([keras.layers.Dense(units=1)])
dnn_model.compile('rmsprop', 'mse')
dnn_model.fit(x_train, y_train, epochs=10)
combined_model = tf.keras.experimental.WideDeepModel(linear_model, dnn_model)
combined_model.compile(optimizer=['sgd', 'adam'],
                       loss=tf.keras.losses.BinaryCrossentropy(),
                       metrics=[tf.keras.metrics.BinaryAccuracy(),
                                tf.keras.metrics.FalseNegatives()])
combined_model.fit([x_train, x_train], y_train, epochs=10)

Train on 627 samples
Epoch 1/10
627/627 [==============================] - 0s 229us/sample - loss: 0.2756
Epoch 2/10
627/627 [==============================] - 0s 81us/sample - loss: 0.2445
Epoch 3/10
 32/627 [>.............................] - ETA: 0s - loss: 0.2150

2022-03-02 10:59:32.528941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:32.552179: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:32.562019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


627/627 [==============================] - 0s 78us/sample - loss: 0.2410
Epoch 4/10
627/627 [==============================] - 0s 78us/sample - loss: 0.2387
Epoch 5/10
627/627 [==============================] - 0s 90us/sample - loss: 0.2375
Epoch 6/10
627/627 [==============================] - 0s 86us/sample - loss: 0.2371
Epoch 7/10
627/627 [==============================] - 0s 98us/sample - loss: 0.2359
Epoch 8/10
627/627 [==============================] - 0s 89us/sample - loss: 0.2355
Epoch 9/10
627/627 [==============================] - 0s 92us/sample - loss: 0.2348
Epoch 10/10
627/627 [==============================] - 0s 78us/sample - loss: 0.2343
Train on 627 samples
Epoch 1/10
627/627 [==============================] - 0s 288us/sample - loss: 1710.4424
Epoch 2/10
 32/627 [>.............................] - ETA: 0s - loss: 1175.6158

2022-03-02 10:59:33.205781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:33.229374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:33.243872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


627/627 [==============================] - 0s 106us/sample - loss: 1602.4912
Epoch 3/10
627/627 [==============================] - 0s 103us/sample - loss: 1507.0347
Epoch 4/10
627/627 [==============================] - 0s 105us/sample - loss: 1417.3558
Epoch 5/10
627/627 [==============================] - 0s 110us/sample - loss: 1336.2586
Epoch 6/10
627/627 [==============================] - 0s 110us/sample - loss: 1256.6302
Epoch 7/10
627/627 [==============================] - 0s 109us/sample - loss: 1179.3949
Epoch 8/10
627/627 [==============================] - 0s 110us/sample - loss: 1107.1923
Epoch 9/10
627/627 [==============================] - 0s 110us/sample - loss: 1034.7015
Epoch 10/10
627/627 [==============================] - 0s 105us/sample - loss: 976.0029
Train on 627 samples
Epoch 1/10
 32/627 [>.............................] - ETA: 2s - loss: 4.3211 - binary_accuracy: 0.7188 - false_negatives_2: 3.0000

2022-03-02 10:59:34.133483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:34.165316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:34.178945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:34.208758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 10:59:34.216595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


627/627 [==============================] - 0s 394us/sample - loss: 4.9784 - binary_accuracy: 0.6635 - false_negatives_2: 94.0000
Epoch 2/10
627/627 [==============================] - 0s 181us/sample - loss: 4.8661 - binary_accuracy: 0.6699 - false_negatives_2: 103.0000
Epoch 3/10
627/627 [==============================] - 0s 182us/sample - loss: 5.9411 - binary_accuracy: 0.6108 - false_negatives_2: 225.0000
Epoch 4/10
627/627 [==============================] - 0s 179us/sample - loss: 5.9781 - binary_accuracy: 0.6124 - false_negatives_2: 243.0000
Epoch 5/10
627/627 [==============================] - 0s 180us/sample - loss: 5.9781 - binary_accuracy: 0.6124 - false_negatives_2: 243.0000
Epoch 6/10
627/627 [==============================] - 0s 175us/sample - loss: 5.9781 - binary_accuracy: 0.6124 - false_negatives_2: 243.0000
Epoch 7/10
627/627 [==============================] - 0s 171us/sample - loss: 5.9781 - binary_accuracy: 0.6124 - false_negatives_2: 243.0000
Epoch 8/10
627/627 [=====

In [38]:
result = combined_model.evaluate([x_eval, x_eval], y_eval)
result = {out: result[i] for i, out in enumerate(combined_model.metrics_names)}
result

{'loss': 5.7843553658687705,
 'binary_accuracy': 0.625,
 'false_negatives_2': 99.0}